In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.tools.tools as sm_tools
import os
os.chdir("..")
os.chdir("C:/Users/Amin/Downloads")
cwd = os.getcwd()

print("Current Working Directory:", cwd)

Current Working Directory: C:\Users\Amin\Downloads


In [2]:
start_index = 0
end_index = 1000

In [3]:
#reading the dataset
dtype_spec = {
    'gvkey': 'string',
}
df = pd.read_csv(f"aggregated_dataset{start_index}_{end_index}.csv", dtype=dtype_spec)
# these columns are redundant or exact copies. For each, There is another column with a different name with the exact same information.
df = df.drop(['date', 'permno', 'permco', 'iid', 'id', 'excntry', 'obs_main',
              'exch_main', 'primary_sec', 'common', 'ret_lag_dif', 'curcd',
              'fx', 'comp_tpci', 'crsp_shrcd','source_crsp','me_company',
              'ret_local','prc_local','me','ret', 'ret_exc_lead1m'
              ], axis=1)
print(len(df))

C:\Users\Amin\AppData\Local\Temp\ipykernel_2824\1646055182.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"aggregated_dataset{start_index}_{end_index}.csv", dtype=dtype_spec)


221699


In [4]:
# I need to calculate 36month betas. So, I can delete stocks that have less than 3 years of data.
group_sizes = df.groupby('gvkey').transform('size')
mask = group_sizes >= 37
filtered_df = df[mask]
print(len(filtered_df))
#converting eom column to datetime format
filtered_df['eom'] = pd.to_datetime(filtered_df['eom'])
filtered_df.head(5)

219360


C:\Users\Amin\AppData\Local\Temp\ipykernel_2824\291929918.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['eom'] = pd.to_datetime(filtered_df['eom'])


,gvkey,eom,size_grp,ret_exc,prc_high,prc_low,bidask,gics,naics,sic,...,dltnetis_mev,dstnetis_mev,dbnetis_mev,netis_mev,fincf_mev,ivol_capm_60m,beta_21d,beta_252d,rvol_252d,rvolhl_21d
0,001000,1970-09-30,micro,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,0.000713,0.213338,0.214051,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,001000,1970-10-31,micro,0.058876,NaN,NaN,0.0,NaN,NaN,NaN,...,0.000679,0.203243,0.203922,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,001000,1970-11-30,micro,NaN,10.875,8.750,0.0,NaN,NaN,3079.0,...,0.000765,0.229035,0.229800,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,001000,1970-12-31,micro,0.107325,10.125,7.625,0.0,NaN,NaN,3079.0,...,0.000706,0.211159,0.211865,NaN,NaN,NaN,0.638698,NaN,NaN,0.034683
4,001000,1971-01-31,micro,-0.103777,10.000,8.750,0.0,NaN,NaN,3079.0,...,0.000765,0.229035,0.229800,NaN,NaN,NaN,4.439515,NaN,NaN,0.024832


In [5]:
mom_df = pd.read_csv("data/F-F_Momentum_Factor.CSV")
ff5_df = pd.read_csv("data/F-F_Research_Data_5_Factors_2x3.CSV")
factor_df = pd.merge(ff5_df,mom_df,how="left").drop(['RF'], axis=1)

factor_df['date'] = factor_df['date'].astype(str)
# Ensure all date strings have the correct length (6 characters)
factor_df['date'] = factor_df['date'].str.zfill(6)  # This ensures leading zeros are added if necessary
# Convert to datetime format (yyyy-mm-dd) with the last day of the month
factor_df['date'] = pd.to_datetime(factor_df['date'] + '01', format='%Y%m%d') + pd.offsets.MonthEnd(0)
factor_df = factor_df.rename(columns={'date': 'eom','Mom   ':"MOM"})
#converting percentage returns to normal returns. 1% to 0.01
factor_df = factor_df.applymap(lambda x: x / 100 if isinstance(x, (int, float)) else x)
factor_df.head(12)


,eom,Mkt-RF,SMB,HML,RMW,CMA,MOM
0,1963-07-31,-0.0039,-0.0041,-0.0097,0.0068,-0.0118,0.0090
1,1963-08-31,0.0507,-0.0080,0.0180,0.0036,-0.0035,0.0101
2,1963-09-30,-0.0157,-0.0052,0.0013,-0.0071,0.0029,0.0019
3,1963-10-31,0.0253,-0.0139,-0.0010,0.0280,-0.0201,0.0312
4,1963-11-30,-0.0085,-0.0088,0.0175,-0.0051,0.0224,-0.0074
5,1963-12-31,0.0183,-0.0210,-0.0002,0.0003,-0.0007,0.0175
6,1964-01-31,0.0224,0.0013,0.0148,0.0017,0.0147,0.0086
7,1964-02-29,0.0154,0.0028,0.0281,-0.0005,0.0091,0.0026
8,1964-03-31,0.0141,0.0123,0.0340,-0.0221,0.0322,0.0075
9,1964-04-30,0.0010,-0.0152,-0.0067,-0.0127,-0.0108,-0.0058


In [6]:
result = filtered_df.merge(factor_df, on='eom', how='left')
result.head(5)

,gvkey,eom,size_grp,ret_exc,prc_high,prc_low,bidask,gics,naics,sic,...,beta_21d,beta_252d,rvol_252d,rvolhl_21d,Mkt-RF,SMB,HML,RMW,CMA,MOM
0,001000,1970-09-30,micro,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0418,0.0851,-0.0547,0.0030,-0.0583,-0.0880
1,001000,1970-10-31,micro,0.058876,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.0228,-0.0443,0.0022,0.0171,0.0234,0.0942
2,001000,1970-11-30,micro,NaN,10.875,8.750,0.0,NaN,NaN,3079.0,...,NaN,NaN,NaN,NaN,0.0460,-0.0386,0.0169,0.0157,0.0147,0.0273
3,001000,1970-12-31,micro,0.107325,10.125,7.625,0.0,NaN,NaN,3079.0,...,0.638698,NaN,NaN,0.034683,0.0572,0.0294,0.0100,0.0027,0.0030,-0.0223
4,001000,1971-01-31,micro,-0.103777,10.000,8.750,0.0,NaN,NaN,3079.0,...,4.439515,NaN,NaN,0.024832,0.0484,0.0754,0.0133,-0.0199,0.0007,-0.0651


In [173]:
def rolling_regression(y, X):
    # Drop any rows where either the dependent or independent variables are NaN
    valid_data = pd.concat([y, X], axis=1).dropna()
    
    # Count the number of observations used in the regression
    num_observations = len(valid_data)
    # sometimes there are some missing values. I will skip beta calculations for stocks that have 6 or more missing values in the last 36 months.
    if num_observations > 30:
        y_clean = valid_data.iloc[:, 0]  # First column is 'y'
        X_clean = valid_data.iloc[:, 1:]  # The rest are 'X'
        X_clean = sm.add_constant(X_clean)  # Add a constant (intercept) to the model
        model = sm.OLS(y_clean, X_clean).fit()
        return model.params, model.tvalues, model.rsquared, num_observations
    else:
        # Return NaNs and zero observations if there's not enough data
        return [np.nan] * (X.shape[1] + 1), [np.nan] * (X.shape[1] + 1), np.nan, 0


temp_df = result[result.gvkey == "002636"]

gvkey=temp_df.gvkey.iloc[0]
temp_df.head(5)

,gvkey,eom,size_grp,ret_exc,prc_high,prc_low,bidask,gics,naics,sic,...,beta_21d,beta_252d,rvol_252d,rvolhl_21d,Mkt-RF,SMB,HML,RMW,CMA,MOM
218322,002636,1974-01-31,nano,-0.006274,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.0017,0.1041,0.0602,-0.0307,0.0442,-0.0885
218323,002636,1974-02-28,nano,-0.058451,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.0047,0.0006,0.0281,-0.0187,0.0263,0.0032
218324,002636,1974-03-31,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.0281,0.0265,-0.0032,0.0280,0.0044,-0.0105
218325,002636,1974-04-30,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.0529,-0.0070,0.0085,0.0287,0.0209,0.0211
218326,002636,1974-05-31,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.0468,-0.0307,-0.0202,0.0495,-0.0042,-0.0039


In [172]:
temp_df.to_csv("temptemp.csv")

In [166]:
gvkey

'002636'

In [174]:
window = 36

# Perform rolling regression
intercepts = []
Mkt_RF_coefs = []
SMB_coefs = []
HML_coefs = []
RMW_coefs = []
CMA_coefs = []
MOM_coefs = []

t_intercepts = []
t_Mkt_RF_coefs = []
t_SMB_coefs = []
t_HML_coefs = []
t_RMW_coefs = []
t_CMA_coefs = []
t_MOM_coefs = []

rsquareds = []
number_of_observations_li = []

eom_li = []

# Rolling regression
for start in range(len(temp_df) - window + 1):
    y = temp_df['ret_exc'].iloc[start:start + window]
    x = temp_df[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'MOM']].iloc[start:start + window]
    
    params, tvalues, rsquared, number_of_obs = rolling_regression(y, x)
    
    intercepts.append(params[0])
    Mkt_RF_coefs.append(params[1])
    SMB_coefs.append(params[2])
    HML_coefs.append(params[3])
    RMW_coefs.append(params[4])
    CMA_coefs.append(params[5])
    MOM_coefs.append(params[6])
    
    t_intercepts.append(tvalues[0])
    t_Mkt_RF_coefs.append(tvalues[1])
    t_SMB_coefs.append(tvalues[2])
    t_HML_coefs.append(tvalues[3])
    t_RMW_coefs.append(tvalues[4])
    t_CMA_coefs.append(tvalues[5])
    t_MOM_coefs.append(tvalues[6])
    rsquareds.append(rsquared)
    number_of_observations_li.append(number_of_obs)

    eom_li.append(temp_df[['eom']].iloc[start + window - 1][0])
# Create the resulting DataFrame
rolling_params_df = pd.DataFrame({
    'Intercept': intercepts,
    'Mkt-RF_Coefficient': Mkt_RF_coefs,
    'SMB_Coefficient': SMB_coefs,
    'HML_Coefficient': HML_coefs,
    'RMW_Coefficient': RMW_coefs,
    'CMA_Coefficient': CMA_coefs,
    'MOM_Coefficient': MOM_coefs,
    't-Intercept': t_intercepts,
    't-Mkt-RF': t_Mkt_RF_coefs,
    't-SMB': t_SMB_coefs,
    't-HML': t_HML_coefs,
    't-RMW': t_RMW_coefs,
    't-CMA': t_CMA_coefs,
    't-MOM': t_MOM_coefs,
    'R-squared': rsquareds,
    'Number_of_Observations': number_of_observations_li,
    "eom": eom_li,
    "gvkey":gvkey
}, index=temp_df.index[window-1:])

rolling_params_df.to_csv("rolling_params_df.csv")


In [175]:
new_temp_df = pd.merge(rolling_params_df,temp_df,how="left",on=["eom","gvkey"])

In [178]:
new_temp_df['realized_alpha_previous_month_betas'] = new_temp_df['ret_exc'] - (new_temp_df["MOM"] * new_temp_df['MOM_Coefficient'].shift(1)) - (new_temp_df["CMA"] * new_temp_df['CMA_Coefficient'].shift(1)) - (new_temp_df["RMW"] * new_temp_df['RMW_Coefficient'].shift(1)) - (new_temp_df["HML"] * new_temp_df['HML_Coefficient'].shift(1)) - (new_temp_df["SMB"] * new_temp_df['SMB_Coefficient'].shift(1)) - (new_temp_df["Mkt-RF"] * new_temp_df['Mkt-RF_Coefficient'].shift(1))

In [179]:
new_temp_df['realized_alpha_same_month_betas'] = new_temp_df['ret_exc'] - (new_temp_df["MOM"] * new_temp_df['MOM_Coefficient']) - (new_temp_df["CMA"] * new_temp_df['CMA_Coefficient']) - (new_temp_df["RMW"] * new_temp_df['RMW_Coefficient']) - (new_temp_df["HML"] * new_temp_df['HML_Coefficient']) - (new_temp_df["SMB"] * new_temp_df['SMB_Coefficient']) - (new_temp_df["Mkt-RF"] * new_temp_df['Mkt-RF_Coefficient'])

In [180]:
print(new_temp_df['ret_exc'].corr(new_temp_df["Mkt-RF"]))
print(new_temp_df['ret_exc'].corr(new_temp_df["realized_alpha_previous_month_betas"]))
print(new_temp_df['ret_exc'].corr(new_temp_df["realized_alpha_same_month_betas"]))
print(new_temp_df["realized_alpha"].corr(new_temp_df["Mkt-RF"]))

0.2397558751354367
0.8029964769949832
0.7952820477831766
-0.09510889758267972


In [181]:
new_temp_df.to_csv("new_temp_df.csv")